# UpdateManagerEntraID

### This notebook shows the step by step of using Powershell Graph SDK (Set-MgUserManagerByRef) to edit/ update the manager of EntraID user.

### First it will show how to export the EntraID user properties from a fiction tenant. Then, it will show how to build a user to object id and a object id to user hash. It will use the hash to record the initial manager status. 

### After that, it will show setting up the table structure and update the user's manager.

## Export user to csv

In [ ]:
# export the EntraID user into users.csv

![export](./exportUsers.jpg)

In [1]:
# assume the filename is users.csv

$u = Import-Csv './users.csv'

In [2]:
$userToIdHash = @{}
$idToUserHash = @{}

$u | ForEach-Object -process {
   $t = @{$_.'Display name' = $_.'Object Id'}
   $userToIdHash += $t

   $t = @{$_.'Object Id' = $_.'Display name'}
   $idToUserHash += $t
}

In [8]:
$userToIdHash


Name                           Value
----                           -----
Alex Wilber                    c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726
Megan Bowen                    c948120e-0b75-4e47-b758-c6c57bddde64
Pradeep Gupta                  9091d05e-e280-4b72-8612-fd8e76ae4795
Joni Sherman                   96ba2e29-c833-4322-83e1-1f609aa3fe4c
P A                            b28254f0-80be-45dd-a1f4-d954e1e1a5f3
Nestor Wilke                   33d9a4ec-2426-4c08-b4f3-97459e49cd3b
Lidia Holloway                 a0df3ed6-b117-4faa-aedf-ab78d0d60ac0
Adele Vance                    e065d899-0464-4a43-b02a-206028b27982
Lee Gu                         98d68316-30d8-43c4-ac35-4319ec3b50e9
Patti Fernandez                ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Lynne Robbins                  27baf846-be3a-4bc0-b382-40d466d82e1b
Isaiah Langer                  f243a1ce-a5e6-42e1-abe1-f49ca450d6a0
Johanna Lorenz                 40d46eb7-4a18-4fa0-b410-c40c92324624
Diego Siciliani                0a02a0f1-4

In [9]:
$idToUserHash


Name                           Value
----                           -----
40d46eb7-4a18-4fa0-b410-c40c9… Johanna Lorenz
e065d899-0464-4a43-b02a-20602… Adele Vance
9091d05e-e280-4b72-8612-fd8e7… Pradeep Gupta
b28254f0-80be-45dd-a1f4-d954e… P A
2c758750-2644-4e13-975f-453e0… Henrietta Mueller
27baf846-be3a-4bc0-b382-40d46… Lynne Robbins
0e799665-9e0b-4226-97c3-694a4… Miriam Graham
96ba2e29-c833-4322-83e1-1f609… Joni Sherman
2ea73151-a011-47b5-9a99-8d37a… Grady Archie
33d9a4ec-2426-4c08-b4f3-97459… Nestor Wilke
98d68316-30d8-43c4-ac35-4319e… Lee Gu
f243a1ce-a5e6-42e1-abe1-f49ca… Isaiah Langer
a0df3ed6-b117-4faa-aedf-ab78d… Lidia Holloway
0a02a0f1-48e1-4010-b904-c7650… Diego Siciliani
c3a3bf8e-d9d4-4b9a-b7d4-17599… Alex Wilber
9a2e7c71-2c59-4b80-ada4-19d85… Daniel Carmack
c948120e-0b75-4e47-b758-c6c57… Megan Bowen
ac4e7475-bf36-4ac3-b8bd-5e0bb… Patti Fernandez



## Record the manager status of the tenant 


In [5]:
Connect-MgGraph

Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.



In [6]:
$users = Get-MgUser

In [7]:
$users | ForEach-Object -Begin {$userManagerArr = @()} `
                        -Process {
                                    $userManagerId = Get-MgUserManager -UserId $_.Id -ErrorAction SilentlyContinue
                                    if ($userManagerId) {
                                        $userManager = $idToUserHash[$userManagerId.Id]
                                    }
                                    else {
                                        $userManager = 'No Manager'
                                    }
                                    $userManagerArr += [PSCustomObject] @{Name = $_.DisplayName;
                                                                            Manager = $userManager;
                                                                            ManagerId = $userManagerId.Id}
                                }

                        

Error: Command failed: SubmitCode: $users | ForEach-Object -Begin {$userManagerArr =  ...

In [10]:
$userManagerArr


Name              Manager         ManagerId
----              -------         ---------
Adele Vance       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Alex Wilber       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Daniel Carmack    No Manager      
Diego Siciliani   Nestor Wilke    33d9a4ec-2426-4c08-b4f3-97459e49cd3b
Grady Archie      Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Henrietta Mueller Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Isaiah Langer     Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Johanna Lorenz    Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Joni Sherman      Nestor Wilke    33d9a4ec-2426-4c08-b4f3-97459e49cd3b
Lee Gu            Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Lidia Holloway    Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Lynne Robbins     Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Megan Bowen       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Miriam Graham     Patti 

In [11]:
# export it to csv so we can restore the manager info later

$userManagerArr | Export-Csv -Path './usermanager.csv' 


## Use Set-MgUserManagerByRef to update user's manager

In [ ]:
# use the exported EntraID user data (users.csv) above and add a column ToManager. ToManager is a dropdown list
# which contains all the users in the organization. Save the file into users.xlsx


In [25]:
$u| Select-Object @{Label='Name'; Expression={$_.'Display Name'}}, @{Label='ObjectID'; Expression={$_.'Object Id'}}, @{Label='Manager'; Expression={$_.Manager}},
                  @{Label='Manager ObjectID'; Expression={$userToIdHash[$_.Manager]}}


Name              ObjectID                             Manager         Manager ObjectID
----              --------                             -------         ----------------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Johanna Lorenz    40d46eb7-4a18-4fa0-b410-c40c92324624 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Joni Sherman      96ba2

Error: Command failed: SubmitCode: $u| Select-Object @{Label='Name'; Expression={$_.' ...

In [28]:
$users


DisplayName       Id                                   Mail                              UserPrinci
                                                                                         palName
-----------       --                                   ----                              ----------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 AdeleV@61b3yr.onmicrosoft.com     AdeleV@61…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 AlexW@61b3yr.onmicrosoft.com      AlexW@61b…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Daniel.C@61b3yr.onmicrosoft.com   Daniel.C@…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 DiegoS@61b3yr.onmicrosoft.com     DiegoS@61…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 GradyA@61b3yr.onmicrosoft.com     GradyA@61…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 HenriettaM@61b3yr.onmicrosoft.com Henrietta…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 IsaiahL@61b3yr.onmicrosoft.com    IsaiahL@6…
Jo